In [2]:
%pwd

'/Users/enitan/Desktop/Projects/Chest-Disease-Classification-MLOPs/research'

In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
from chest_disease_classification import logger
from chest_disease_classification.utils import read_yaml, create_directory
import gdown
import zipfile


In [6]:
from pathlib import Path
from dataclasses import dataclass
from pydantic import BaseModel

In [16]:

class ModelTrainingConfig(BaseModel):
    root_dir: Path
    model_dir: Path
    updated_base_model_dir: Path
    training_data: Path
    epoch: int
    batch_size: int
    augmentation: bool
    image_size: list
    learning_rate: float


In [17]:
class ConfigurationManager:
    def __init__(self, config_file_path, params_file_path):
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path

        self.config = read_yaml(self.config_file_path)
        self.params = read_yaml(self.params_file_path)

        create_directory(self.config.artifact_root)
    
    def get_training_config(self) -> ModelTrainingConfig:
        config_training = self.config.model_training
        config_base_model = self.config.base_model
        config_data_ingestion = self.config.data_ingestion
        training_data = os.path.join(config_data_ingestion.unzip_dir, "Chest-CT-Scan-data")


        create_directory(config_training.root_dir)
        model_training_config = ModelTrainingConfig(
            root_dir = Path(config_training.root_dir),
            model_dir = Path(config_training.model_dir),
            updated_base_model_dir = Path(config_base_model.updated_base_model_dir),
            training_data = Path(training_data),
            epoch = self.params.epoch,
            batch_size = self.params.batch_size,
            augmentation = self.params.augmentation,
            image_size = self.params.image_size,
            learning_rate = self.params.learning_rate
        )
        return model_training_config
        



In [9]:

import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())


2.16.2
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:
import tensorflow as tf

In [18]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def get_base_model(self):
        model = tf.keras.models.load_model(self.config.updated_base_model_dir)
        model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate = self.config.learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"],
            run_eagerly=True,
        )

        return model
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict( rescale = 1./255, validation_split = 0.2)

        data_flow_kwargs = dict(target_size = self.config.image_size[:-1], batch_size = self.config.batch_size, interpolation = "bilinear")
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **data_flow_kwargs
        )

        if self.config.augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            shuffle = True,
            subset = "training",
            **data_flow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        model = self.get_base_model()
        model.fit(
            self.train_generator,
            epochs = self.config.epoch,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(path = self.config.model_dir, model = model)

In [ ]:
config = ConfigurationManager(config_file_path = Path("config/config.yaml"), params_file_path = Path("params.yaml"))
training_config = config.get_training_config()
model_training = ModelTraining(config = training_config)
model_training.get_base_model()
model_training.train_valid_generator()
model_training.train()

[2025-12-04 03:57:02,622: INFO: utils: parameters successfully loaded from, config/config.yaml]
[2025-12-04 03:57:02,625: INFO: utils: parameters successfully loaded from, params.yaml]
[2025-12-04 03:57:02,626: INFO: utils: Directory created at: artifact]
[2025-12-04 03:57:02,627: INFO: utils: Directory created at: artifact/trained_model]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
[2025-12-04 03:57:02,865: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 512ms/step - accuracy: 0.5328 - loss: 18.8647 - val_accuracy: 0.3906 - val_loss: 20.1953
[2025-12-04 03:57:12,231: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_m

In [ ]:

print(tf.__version__)
print("Eager:", tf.executing_eagerly())
